#Installing Dependencies

In [1]:
!pip install onn

  Stored in directory: /root/.cache/pip/wheels/71/9f/46/14b8f23801d65ea3edd4eb9a165651526d4f12ff1d6efe8257
Successfully built onn


##Importing Dependencies

In [0]:
from onn.OnlineNeuralNetwork import ONN
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

## Initializing Network

In [0]:
onn_network = ONN(features_size=10, max_num_hidden_layers=5, qtd_neuron_per_hidden_layer=40, n_classes=10)

##Creating Fake Classification Dataset

In [0]:
X, Y = make_classification(n_samples=50000, n_features=10, n_informative=4, n_redundant=0, n_classes=10,
                           n_clusters_per_class=1, class_sep=3)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42, shuffle=True)

##Learning and predicting at the same time

In [26]:
for i in range(len(X_train)):
  onn_network.partial_fit(np.asarray([X_train[i, :]]), np.asarray([y_train[i]]))
  
  if i % 1000 == 0:
    predictions = onn_network.predict(X_test)
    print("Online Accuracy: {}".format(accuracy_score(y_test, predictions)))

Online Accuracy: 0.05466666666666667
Alpha:[0.83948636 0.04012838 0.04012838 0.04012838 0.04012838]
Training Loss: 1.3012835
Online Accuracy: 0.9549333333333333
Alpha:[0.8393235 0.0401691 0.0401691 0.0401691 0.0401691]
Training Loss: 0.43253973
Online Accuracy: 0.9649333333333333
Alpha:[0.8398507 0.0400373 0.0400373 0.0400373 0.0400373]
Training Loss: 0.35385495
Online Accuracy: 0.9657333333333333
Alpha:[0.8398936  0.04002658 0.04002658 0.04002658 0.04002658]
Training Loss: 0.26460257
Online Accuracy: 0.9705333333333334
Alpha:[0.83983797 0.04004049 0.04004049 0.04004049 0.04004049]
Training Loss: 0.2707248
Online Accuracy: 0.9728666666666667
Alpha:[0.8399278  0.04001803 0.04001803 0.04001803 0.04001803]
Training Loss: 0.23593011
Online Accuracy: 0.9732666666666666
Alpha:[0.83984274 0.04003929 0.04003929 0.04003929 0.04003929]
Training Loss: 0.22243583
Online Accuracy: 0.9753333333333334
Alpha:[0.83947986 0.04013    0.04013    0.04013    0.04013   ]
Training Loss: 0.22021466
Online Accu